In [1]:
!pip install simpy

In [2]:
import simpy
import random
import numpy as np

In [3]:
env = simpy.Environment()

In [4]:
server = simpy.Resource(env, capacity=1)

In [8]:
import simpy
import random
import numpy as np

def run_simulation(arrival_rate, service_rate, sim_time):
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=1)

    wait_times = []

    def customer(env):
        arrival = env.now
        with server.request() as req:
            yield req
            wait_times.append(env.now - arrival)
            service_time = random.expovariate(service_rate)
            yield env.timeout(service_time)

    def generator(env):
        while True:
            yield env.timeout(random.expovariate(arrival_rate))
            env.process(customer(env))

    env.process(generator(env))
    env.run(until=sim_time)

    return np.mean(wait_times)

In [9]:
import pandas as pd

data = []

for _ in range(1000):
    arrival_rate = random.uniform(0.1, 5.0)
    service_rate = random.uniform(0.5, 10.0)
    sim_time = random.randint(200, 800)

    avg_wait = run_simulation(arrival_rate, service_rate, sim_time)

    data.append([
        arrival_rate,
        service_rate,
        sim_time,
        avg_wait
    ])

df = pd.DataFrame(data, columns=[
    "arrival_rate",
    "service_rate",
    "simulation_time",
    "avg_waiting_time"
])

df.head()

,arrival_rate,service_rate,simulation_time,avg_waiting_time
0,4.355526,7.436171,619,0.156452
1,0.598634,5.584738,569,0.023106
2,2.377151,8.272574,275,0.050397
3,1.595138,5.356611,732,0.065898
4,0.445711,1.556963,505,0.231302


In [10]:
from sklearn.model_selection import train_test_split

X = df[["arrival_rate", "service_rate", "simulation_time"]]
y = df["avg_waiting_time"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "SVR": SVR(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)

    results.append({
        "Model": name,
        "RMSE": rmse,
        "R2 Score": r2_score(y_test, preds)
    })

results_df = pd.DataFrame(results)
results_df

,Model,RMSE,R2 Score
0,Linear Regression,42.050328,0.414407
1,Ridge Regression,42.050958,0.414389
2,Lasso Regression,42.103228,0.412933
3,SVR,59.330315,-0.165763
4,Random Forest,11.617745,0.955301
5,Gradient Boosting,12.088278,0.951607
